## Overview

This notebook illustrates how to use [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview) with [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview) as a vector database.

The wordy [official documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/use-vertexai-vector-search).

For more details on RAG corpus/file management check out [Vertex AI RAG Engine API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api)

# SETUP STEPS

**Before running this notebook**, install `pip` and `ipykernel` in your virtual environment. Run the below command in the same folder that holds your `.venv`.

```bash
uv pip install --python .venv/bin/python pip ipykernel ipython
```

Then select this environment as your notebook kernel and run the cell below to install dependencies.

Also make sure you have the gcloud CLI tool and that you run `gcloud auth application-default login` to login to GCP with the correct account credentials.  

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform google-genai dotenv

Note: you may need to restart the kernel to use updated packages.



__Note:__ You may need to restart the kernel/jupyter runtime for this to work - after the install step above and after running.

Check also that you're connected to the local .venv

In [2]:
import sys
print(sys.executable)

/Users/zubinpratap/Documents/myprojects/research-public/.venv/bin/python


Check that your Google Application Default Credentials are working right.

In [3]:
from google.auth import default
from google.auth.exceptions import RefreshError

# Test authentication first
try:
    credentials, project = default()
    print(f"✓ Authentication successful. Project: {project}")
    print(f"✓ Credentials type: {type(credentials).__name__}")
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    print("\nTry running: gcloud auth application-default login")
    raise

✓ Authentication successful. Project: vertexai-gdrive-rag
✓ Credentials type: Credentials


In [4]:
# Get all Env Vars
from dotenv import load_dotenv
import os
load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT_ID")
LOCATION = os.getenv("GCP_RAG_CORPUS_LOCATION")
CORPUS_NAME = os.getenv("CORPUS_NAME")
if not PROJECT_ID:
    print("Missing Project ID")
if not LOCATION:
    print("Missing RAG corpus location")
if not CORPUS_NAME:
    print("Missing RAG corpus Name")

POD_TRANSCRIPTS_DIR = "https://drive.google.com/drive/folders/1Ouyv9pMem_GNczS9I9pD-WUQ3pM3ZXVN"
MFM_TRANSCRIPTS_DIR = "https://drive.google.com/drive/folders/1F85NFT4wPVuxHHn-UwmvvpAsUvuzNKtj"

# Vertex AI RAG engine accepts Google Cloud Storage and Google Drive Links
PATHS = [POD_TRANSCRIPTS_DIR , MFM_TRANSCRIPTS_DIR]

In [5]:
# Set quota project to avoid warnings
os.environ['GOOGLE_CLOUD_QUOTA_PROJECT'] = PROJECT_ID

from vertexai import rag
import vertexai

# Get Corpus Names
# Initialize Vertex AI first (needed for API calls)
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Try to list corpora, but if it fails, construct the corpus name directly
try:
    corpora = rag.list_corpora()
    # print(f"Found  corpora: {corpora}") 
    for corpus in corpora:
        print(f" corpus name: - {corpus.name}")
        if CORPUS_NAME in corpus.name:
            print(f"✓ Using full corpus name: {CORPUS_NAME}")
            break
    else:
        print(f"⚠ Corpus '{CORPUS_NAME}' not found. You may need to create it first.")
        CORPUS_NAME = f"{parent}/ragCorpora/{CORPUS_NAME}"
        print(f"  Will use: {CORPUS_NAME}")
except Exception as e:
    print(f"⚠ Could not list corpora: {e}")

 corpus name: - projects/vertexai-gdrive-rag/locations/asia-northeast1/ragCorpora/2305843009213693952
✓ Using full corpus name: projects/vertexai-gdrive-rag/locations/asia-northeast1/ragCorpora/2305843009213693952


In [6]:
# This code is taken from https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-rag-import-files

# Populate the Corpus
response = rag.import_files(
    corpus_name=CORPUS_NAME,
    paths=PATHS,
    transformation_config=rag.TransformationConfig(
        rag.ChunkingConfig(chunk_size=512, chunk_overlap=100)
    ),
    # import_result_sink="gs://sample-existing-folder/sample_import_result_unique.ndjson",  # Optional, commented out for now
    max_embedding_requests_per_min=900,  # Optional
)
print(f"✓ Imported {response.imported_rag_files_count} files.")
print(f"Total files in Corpus: {len(list(rag.list_files(CORPUS_NAME)))}")

✓ Imported 0 files.
Total files in Corpus: 95
